# 검증

### 유사암진단비 외 담보

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from MyPV import MyPV

In [2]:
# 검증 모듈
mypv = MyPV('실속있는보장보험_STD.xlsx')

## 검증대상건

    P, V, ST 테이블을 서로 Join한 테이블 작성

In [3]:
# 검증대상건
Tab_check = pd.read_csv('Preprocessed.csv', encoding='cp949')

# 유사암진단비(LB4648)는 샘플로 따로 짬
Tab_check = Tab_check.loc[Tab_check['담보코드'] != 'LB4648']        

# string ---> int
for col in (Tab_check.columns):
    if col not in ['담보코드', '보험기간구분','표준해약공제액']:
        Tab_check[col] = Tab_check[col].apply(lambda x:int(x) if x!="^" else x)
Tab_check.head()

,담보코드,보험기간,납입기간,보험기간구분,납입주기,성별,연령,상해급수,운전자급수,가입금액,...,Pr,G,V(0),V(1),V(2),V(3),V(4),V(5),신계약비,표준해약공제액
0,LB7408,5,5,A080,412,1,30,^,^,1000000,...,5,14,0,15,25,26,18,0,388,42.761583
1,LB7408,5,5,A080,412,1,31,^,^,1000000,...,6,15,0,20,30,31,23,0,391,49.220016
2,LB7408,5,5,A080,412,1,32,^,^,1000000,...,7,16,0,20,32,34,22,0,394,56.378872
3,LB7408,5,5,A080,412,1,33,^,^,1000000,...,8,18,0,23,38,38,29,0,397,64.794548
4,LB7408,5,5,A080,412,1,34,^,^,1000000,...,9,19,0,27,40,44,30,0,401,73.878022


##  검증 Loop

오류없이 루프가 다 돌면 됨

In [5]:
with open('result.csv', 'w') as f:    
    for row in tqdm(Tab_check[['담보코드', '보험기간', '납입기간', '성별', '연령', '상해급수', '운전자급수', \
        '가입금액', 'NP', 'G', 'V(0)', 'V(1)', 'V(2)', 'V(3)', 'V(4)', 'V(5)', '신계약비', '표준해약공제액']].values):

        # unpack
        cov, n, m, sex, x, injure, driver, AMT, NP12, G12, V0, V1, V2, V3, V4, V5, newContractFee, std_alpha = row
        tVx = [V if V=="^" else int(V) for V in [V0, V1, V2, V3, V4, V5]]

        sex = int(sex)
        n = int(n)
        m = int(m)
        AMT = int(AMT)

        try:
            injure = int(injure)
        except:
            pass
        try:
            driver = int(driver)
        except:
            pass

        if cov == "LA1568":         # 대중교통이용중교통상해사망
            injure, driver = 0, 0

        if cov == "LA9218" and driver in [3, '3']:          # 교통상해사망 3급인 경우 위험률 코드를 달리 설정함
            cov = "LA9218_sub"

   
        mypv.setArgs(coverage=cov, sex=sex, x=x, n=n, m=m, injure=injure,driver=driver, AMT=AMT)

        # 사업비 세팅
        mypv.setExpense()
        # 계산결과
        result = mypv.CalcPV()
        my_NP1 = result['순보험료'][0]
        my_NP12 = result['순보험료'][-1]
        my_G1 = result['영업보험료'][0]
        my_G12 = result['영업보험료'][-1]
        my_tVx = result['tVx']
        my_newContractFee = result['신계약비']
        
        # 계산결과 틀리면 ERROR 발생시키기
        if NP12 != round(my_NP12 * AMT):        
            raise Exception(f'ERROR : {cov}담보 - 순보험료 오류 | {NP12} & {my_NP12*AMT}')
            
        if G12 != round(my_G12 * AMT):            
            raise Exception(f'ERROR : {cov}담보 - 영업보험료 오류| {G12} & {my_G12*AMT}')

        if tVx != my_tVx:        
            raise Exception(f'ERROR : {cov}담보 - 준비금 오류')
    
        f.write(f"{my_NP1},{my_NP12},{my_G1},{my_G12},{tVx[0]},{tVx[1]},{tVx[2]},{tVx[3]},{tVx[4]},{tVx[5]},{mypv.alpha1},{mypv.alpha3},{AMT}\n")

100%|██████████| 16340/16340 [11:15<00:00, 24.19it/s]


In [5]:
cov = "LB5758"
sex = 1
n = 5
m = 5
x = 40
AMT = 1000000

In [8]:
mypv.setArgs(coverage=cov, sex=sex, x=x, n=n, m=m,  AMT=AMT)
mypv.setExpense()

In [9]:
mypv.CalcPV()

{'납방계수': [1, 2, 3, 4, 12],
 '순보험료': [0.0007965269619858155,
  0.00040078612879927613,
  0.00026775608539555837,
  0.00020102973741928227,
  6.715214613700099e-05],
 '영업보험료': [0.0014091963022283911,
  0.0007093772785555323,
  0.0004739898404475141,
  0.0003558956184001624,
  0.0001189016150492484],
 '신계약비': 1808,
 'tVx': [0, 207, 324, 335, 230, 0],
 '표준해약공제액': 199.84652419}

In [11]:
mypv.CalcSymbol(True)

,qx_exit(0),qx_exit(1),qx_exit(99),qx_benefit(1),lx(0),lx(1),l'x,Dx,D'x,Nx,N'x,Cx(1),Mx(1),SUMx
0,0.000602,0.000602,0.000602,0.000602,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,563573.908215,563573.908215,59.461322,479.731310,378.785806
1,0.000696,0.000696,0.000696,0.000696,99939.800000,99939.800000,99939.800000,97502.243902,97502.243902,463573.908215,463573.908215,67.028873,420.269989,319.324484
2,0.000804,0.000804,0.000804,0.000804,99870.241899,99870.241899,99870.241899,95057.933991,95057.933991,366071.664312,366071.664312,75.488794,353.241115,252.295611
3,0.000919,0.000919,0.000919,0.000919,99789.946225,99789.946225,99789.946225,92664.885280,92664.885280,271013.730321,271013.730321,84.114094,277.752322,176.806817
4,0.001039,0.001039,0.001039,0.001039,99698.239264,99698.239264,99698.239264,90321.684147,90321.684147,178348.845041,178348.845041,92.692723,193.638228,92.692723
5,0.001161,0.001161,0.001161,0.001161,99594.652794,99594.652794,99594.652794,88027.160895,88027.160895,88027.160895,88027.160895,100.945504,100.945504,0.000000


In [12]:
up = 378.785806
down = 12*(563573.908215 - 88027.160895 - 11/24*(100000.000000 - 88027.160895))
up/down

6.71521461502614e-05

In [13]:
down

5640710.3527625